In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        import warnings
warnings.filterwarnings('ignore', message='invalid value encountered')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os

offer_metadata_df = pd.read_parquet('/kaggle/input/bitblenders-007/offer_metadata.parquet')
offer_metadata_df


In [ ]:
offer_metadatar_df = pd.read_csv('/kaggle/input/nothin/offer_metadata_cleaned_revised.csv')
offer_metadatar_df

In [ ]:
# Replace 'UNKNOWN' with NaN for proper handling
offer_metadatar_df['f374'] = offer_metadatar_df['f374'].replace('UNKNOWN', np.nan)
offer_metadatar_df

In [ ]:
offer_metadatar_df['id12'] = pd.to_datetime(offer_metadatar_df['id12'])
offer_metadatar_df['id13'] = pd.to_datetime(offer_metadatar_df['id13'])
offer_metadatar_df

In [ ]:
offer_metadatar_df['f378'] = offer_metadatar_df['f378'].replace('UNKNOWN', np.nan)
offer_metadatar_df

In [ ]:
import pandas as pd
import numpy as np

def create_offer_duration_categories(offer_metadata_df):
    """
    Create categorical duration features for offer metadata
    """
    # Make a copy to avoid modifying original dataframe
    df = offer_metadatar_df.copy()
    
    # Ensure date columns are datetime
    df['id12'] = pd.to_datetime(df['id12'])  # Start date
    df['id13'] = pd.to_datetime(df['id13'])  # End date
    
    # Calculate duration in days
    df['duration_days'] = (df['id13'] - df['id12']).dt.days
    
    # Create offer duration categories
    def categorize_duration(row):
        duration = row['duration_days']
        end_year = row['id13'].year
        
        # Handle permanent offers (far future dates)
        if end_year >= 2099:
            return 'permanent'
        
        # Handle negative durations (data quality issue)
        if duration < 0:
            return 'invalid'
        
        # Categorize based on duration
        if duration <= 1:
            return 'flash_sale'      # Same day or 1 day
        elif duration <= 7:
            return 'weekly'          # Up to 1 week
        elif duration <= 30:
            return 'short_term'      # Up to 1 month
        elif duration <= 90:
            return 'quarterly'       # Up to 3 months
        elif duration <= 365:
            return 'seasonal'        # Up to 1 year
        else:
            return 'long_term'       # More than 1 year
    
    # Apply categorization
    df['offer_duration_category'] = df.apply(categorize_duration, axis=1)
    
    # Create additional useful features
    df['is_permanent_offer'] = (df['offer_duration_category'] == 'permanent').astype(int)
    df['is_flash_sale'] = (df['offer_duration_category'] == 'flash_sale').astype(int)
    
    # Summary statistics
    print("Offer Duration Category Distribution:")
    print("="*40)
    category_counts = df['offer_duration_category'].value_counts()
    for category, count in category_counts.items():
        percentage = (count / len(df)) * 100
        print(f"{category:15}: {count:5d} ({percentage:5.1f}%)")
    
    print(f"\nTotal offers: {len(df)}")
    
    # Show average discount by category
    print("\nAverage Discount Rate by Duration Category:")
    print("="*45)
    discount_by_category = df.groupby('offer_duration_category')['f376'].agg(['mean', 'count'])
    for category in discount_by_category.index:
        avg_discount = discount_by_category.loc[category, 'mean']
        count = discount_by_category.loc[category, 'count']
        print(f"{category:15}: {avg_discount:6.2f}% (n={count})")
    
    # Check for data quality issues
    invalid_durations = df[df['offer_duration_category'] == 'invalid']
    if len(invalid_durations) > 0:
        print(f"\n⚠️  Warning: {len(invalid_durations)} offers have invalid durations (end before start)")
        print("Sample invalid offers:")
        print(invalid_durations[['id3', 'id9', 'id12', 'id13', 'duration_days']].head())
    
    return df

# Alternative function with more business-focused categories
def create_business_duration_categories(offer_metadata_df):
    """
    Create business-focused duration categories
    """
    df = offer_metadatar_df.copy()
    
    # Ensure date columns are datetime
    df['id12'] = pd.to_datetime(df['id12'])
    df['id13'] = pd.to_datetime(df['id13'])
    df['duration_days'] = (df['id13'] - df['id12']).dt.days
    
    # Business-focused categorization
    conditions = [
        (df['id13'].dt.year >= 2099),                           # Permanent offers
        (df['duration_days'] < 0),                              # Invalid
        (df['duration_days'] <= 1),                             # Flash sales
        (df['duration_days'] <= 7),                             # Limited time
        (df['duration_days'] <= 30),                            # Monthly campaigns
        (df['duration_days'] <= 365),                           # Long campaigns
        (df['duration_days'] > 365)                             # Extended offers
    ]
    
    choices = [
        'permanent_offer',
        'invalid_duration', 
        'flash_sale',
        'limited_time',
        'monthly_campaign',
        'long_campaign',
        'extended_offer'
    ]
    
    df['business_duration_type'] = np.select(conditions, choices, default='unknown')
    
    return df

# Usage example:
# offer_metadata_df = create_offer_duration_categories(offer_metadata_df)

# If you want to see just the new columns:
def show_duration_analysis(offer_metadatar_df):
    """
    Display analysis of duration categories
    """
    print("Sample of new duration categories:")
    print(offer_metadatar_df[['id3', 'id9', 'duration_days', 'offer_duration_category', 'f376']].head(10))
    
    print("\nDuration vs Discount Analysis:")
    pivot_table = pd.crosstab(
        offer_metadatar_df['offer_duration_category'], 
        offer_metadatar_df['has_discount'],
        margins=True
    )
    print(pivot_table)

# Run the main function:
# offer_metadata_df = create_offer_duration_categories(offer_metadata_df)
offer_metadatar_df

In [ ]:
# Run the function and assign the result back to your dataframe
offer_metadatar_df = create_offer_duration_categories(offer_metadata_df)

# Now check if the column exists
print("Columns in dataframe:")
print(offer_metadatar_df.columns.tolist())

# View the new column
print("\nFirst few rows with new duration category:")
print(offer_metadatar_df[['id3', 'id9', 'duration_days', 'offer_duration_category']].head())

In [ ]:
# Step 1: Check if the function ran
print("Shape before:", offer_metadatar_df.shape)

# Step 2: Run function with explicit assignment
offer_metadatar_df = create_offer_duration_categories(offer_metadata_df)

# Step 3: Check shape after
print("Shape after:", offer_metadatar_df.shape)

# Step 4: Look for the specific column
if 'offer_duration_category' in offer_metadata_df.columns:
    print("✓ Column created successfully!")
    print(offer_metadatar_df['offer_duration_category'].value_counts())
else:
    print("✗ Column not found. Available columns:")
    print(offer_metadatar_df.columns.tolist())

In [ ]:
offer_metadata_df

In [ ]:
# Remove the two columns
offer_metadatar_df = offer_metadatar_df.drop(['is_permanent_offer', 'is_flash_sale'], axis=1)
offer_metadatar_df

In [ ]:
offer_metadatar_df.to_parquet('offer_metadata70.parquet', index=False)
offer_metadatar_df.to_parquet('/kaggle/working/offer_metadata69.parquet', index=False)

print("✓ File saved to output directory!")
print("Go to 'Output' tab on the right panel to download")

In [ ]:
offer_metadatar_df['f378'] = offer_metadatar_df['f378'].replace(['N', 'n', 'NA'], np.nan)
offer_metadatar_df

In [ ]:
offer_metadatar_df.to_parquet('offer_metadata71.parquet', index=False)
offer_metadatar_df.to_parquet('/kaggle/working/offer_metadata71.parquet', index=False)